In [62]:
# XGBsurv benchmark
from xgbsurv.datasets import load_metabric, load_flchain, load_rgbsg, load_support
#from xgbsurv.datasets import (load_flchain, load_rgbsg, load_support,
#load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
from numpy import savetxt
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.model_selection import RandomizedSearchCV, KFold, StratifiedKFold
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, LabelEncoder,LabelBinarizer
# import models
from pycox.evaluation import EvalSurv
from xgbsurv.models.utils import transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import sys
#sys.path.append('/Users/JUSC/Documents/xgbsurv_benchmarking/deep_learning/')
from loss_functions_pytorch import EHLoss, eh_likelihood_torch_2, AFTLoss
from skorch import NeuralNet
from skorch.callbacks import EarlyStopping
from skorch.dataset import CVSplit, ValidSplit
import torch
from torch import nn
import torch.nn.init as init

## Set Parameters
deep learning hyperparameter spaces follow pycox paper
Time-to-Event Prediction with Neural Networks and Cox Regression
page 21

In [2]:
# set parameters, put into function
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10 # set to 50
#n_iter_cind = 200
early_stopping_rounds=10
base_score = 0.0

param_grid_breslow = {
    'estimator__module__n_layers': [1, 2, 4],
    'estimator__module__n_layers': [1, 2, 4],
    'estimator__module__num_nodes': [64, 128, 256, 512],
    'estimator__module__dropout': scuniform(0.0,0.7),
    'estimator__optimizer__weight_decay': [0.4, 0.2, 0.1, 0.05, 0.02, 0.01, 0],
    'estimator__batch_size': [64, 128, 256, 512, 1024],
    #lr not in paper because of learning rate finder
    # note: setting learning rate higher would make exp(partial_hazard) explode
    'estimator__lr': scloguniform(0.001,0.01),
    #'max_epochs':  scrandint(10,20), # corresponds to num_rounds
}

## Set MLP Model

In [3]:

class SurvivalModel(nn.Module):
    def __init__(self, n_layers, in_features, num_nodes, dropout, out_features):
        super(SurvivalModel, self).__init__()
        self.n_layers = n_layers
        self.in_features = in_features
        self.num_nodes = num_nodes
        self.dropout = dropout
        self.out_features = out_features
        model = []
        # first layer
        model.append(torch.nn.Linear(in_features, num_nodes))
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Dropout(dropout))
        model.append(torch.nn.BatchNorm1d(num_nodes))

        for i in range(n_layers-1):
            model.append(torch.nn.Linear(num_nodes, num_nodes))
            #init.kaiming_normal_(model[-1].weight, nonlinearity='relu')
            model.append(torch.nn.ReLU())
            model.append(torch.nn.Dropout(dropout))
            model.append(torch.nn.BatchNorm1d(num_nodes))

        # output layer
        model.append(torch.nn.Linear(num_nodes, out_features))
    
        self.layers = nn.Sequential(*model)

        # for layer in self.layers:
        #     if isinstance(layer, nn.Linear):
        #         #nn.init.uniform_(layer.weight, a=-0.5, b=0.5)
        #         nn.init.kaiming_normal_(layer.weight)


    def forward(self, x):
        res = self.layers(x)
        print('res',res)
        return res


## Set Splitting Strategy

In [4]:
# Define stratified inner k-fold cross-validation
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        if isinstance(y,pd.DataFrame):
            y = y.to_numpy()
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        print(super().split(X, bins, groups=groups))
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)
inner_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)



## Set Scoring Function

In [5]:
# Define Scorer
def custom_scoring_function(y_true, y_pred):

        #y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, np.ndarray):
            y_pred = torch.from_numpy(y_pred)
        if isinstance(y_true, np.ndarray):
            y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, pd.Series):
            y_pred = torch.tensor(y_pred.values)
        if isinstance(y_true, pd.Series):
            y_true = torch.tensor(y_true.values)
        score = eh_likelihood_torch_2(y_true, y_pred)
        print('score',score)
        return score.numpy()

scoring_function = make_scorer(custom_scoring_function, greater_is_better=False)

## Set Data Loading Functions

In [6]:
# adapt this for bool case
class CustomStandardScaler(StandardScaler):
    
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super().__init__(copy=copy, with_mean=with_mean, with_std=with_std)
        
    def fit(self, X, y=None):
        # Add your own code here
        return super().fit(X, y)
    
    def transform(self, X, y=None):
        # Add your own code here
        X_transformed = super().transform(X, y)
        # Add your own code here
        return X_transformed.astype(np.float32)
    
    def fit_transform(self, X, y=None):
        # Add your own code here
        
        #to_add = X[X.columns[X.columns.isin(['sex','D'])]]
        #X = X[X.columns[~X.columns.isin(['sex','D'])]]
        #df.columns.get_loc('age')
        X_transformed = super().fit_transform(X, y)
        #to_add = to_add.values.reshape(-1, 1)

        # Horizontally stack the two arrays
        #result = np.hstack((X_transformed, to_add))
        
        # Add your own code here
        return X_transformed.astype(np.float32)

class CustomStandardScaler2(StandardScaler):
    """Just to change the datatype of bool variables."""
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super().__init__(copy=copy, with_mean=with_mean, with_std=with_std)
        
    def fit(self, X, y=None):
        # Add your own code here
        return super().fit(X, y)
    
    def transform(self, X, y=None):
        # Add your own code here
        X_transformed = super().transform(X, y)
        # Add your own code here
        return X_transformed.astype(np.float32)
    
    def fit_transform(self, X, y=None):
        # Add your own code here
        
        to_return = X
        #X = X[X.columns[~X.columns.isin(['sex','D'])]]
        #df.columns.get_loc('age')
        X_transformed = super().fit_transform(X, y)
        #to_add = to_add.values.reshape(-1, 1)

        # Horizontally stack the two arrays
        #result = np.hstack((X_transformed, to_add))
        
        # Add your own code here
        return to_return.astype(np.float32)
    
class CustomLabelBinarizer(LabelBinarizer):
    def __init__(self, neg_label=0, pos_label=1, sparse_output=False):
        super().__init__(neg_label=0, pos_label=1, sparse_output=False)

    def fit(self, X, y=None):
        # Add custom fit logic here
        # Call the parent class's fit method
        print('fit',X)
        super().fit(X, y)

        return self

    def transform(self, X, y=None):
        # Add custom transform logic here

        # Call the parent class's transform method
        print('transform',X)
        transformed_X = super().transform(X)

        return transformed_X
    
    def fit_transform(self, X, y=None):
        # Add custom transform logic here

        # Call the parent class's transform method
        print(y)
        print('fit_transform',X.values)
        to_add = X[X.columns[X.columns.isin(['sex','D'])]]
        X = X[X.columns[~X.columns.isin(['sex','D'])]]
        transformed_X = super().fit_transform(X.values)
    
        return transformed_X.astype(np.float32)

    def inverse_transform(self, X, y=None):
        # Add custom inverse_transform logic here

        # Call the parent class's inverse_transform method
        inverse_X = super().inverse_transform(X)

        return inverse_X.astype(np.float32)
    
# ct = make_column_transformer(
#         (CustomStandardScaler(), make_column_selector(dtype_include=['float32'])),
#         #(CustomLabelBinarizer(), make_column_selector(dtype_include=['bool'])), remainder='passthrough')

ct = CustomStandardScaler()
data = load_flchain(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
X  = data.data
X = ct.fit_transform(X)
X.shape

(7871, 8)

## Set Training Function

In [7]:
# set parameters, put into function
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 1 # # set to 50
early_stopping_rounds=10
base_score = 0.0

def train_eval(X, y, net, n_iter, filename):

        dataset_name = filename.split('_')[0]
        # add IBS later
        outer_scores = {'cindex_train_'+dataset_name:[], 'cindex_test_'+dataset_name:[]}
        best_params = {'best_params_'+dataset_name:[]}
        best_model = {'best_model_'+dataset_name:[]}
        # ct = make_column_transformer(
        #  (StandardScaler(),
        #  make_column_selector(dtype_include=float,dtype_exclude=bool)),)
        # (LabelBinarizer(),
        # make_column_selector(dtype_include=bool, dtype_exclude=float)))
        ct = make_column_transformer(
        (CustomStandardScaler(), make_column_selector(dtype_include=['float32'])),
        (CustomStandardScaler2(), make_column_selector(dtype_include=['category'])), remainder='passthrough')
        #columns_to_transform = [col for col in X.columns if col != 'sex']
        #ct = make_column_transformer(
        #(StandardScaler(), columns_to_transform), 
        #(LabelBinarizer(), make_column_selector(dtype_include='bool')),
        #remainder='passthrough'
        #)
        pipe = Pipeline(
        [('scaler', ct),
        ('estimator', net)]
        )
        # pipe = Pipeline([
        # ('scale', StandardScaler()),
        # ('estimator', net),
        # ])
        rs = RandomizedSearchCV(pipe, param_grid_breslow, scoring = scoring_function, n_jobs=-1, 
                            cv=inner_custom_cv, n_iter=n_iter, refit=True)
        
        for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
                # Split data into training and testing sets for outer fold
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                print(X_train.shape, type(X_train))
                print(y_train.shape, type(y_train))
                print(X_test.shape, X_test.shape)
                print(y_test.shape, y_test.shape)
                # save splits and data
                savetxt('splits/train_index_'+str(i)+'_'+filename, train_index, delimiter=',')
                savetxt('splits/test_index_'+str(i)+'_'+filename, test_index, delimiter=',')
                
                savetxt('splits/X_train_'+str(i)+'_'+filename, X_train, delimiter=',')
                savetxt('splits/X_test_'+str(i)+'_'+filename, X_test, delimiter=',')

                savetxt('splits/y_train_'+str(i)+'_'+filename, y_train, delimiter=',')
                savetxt('splits/y_test_'+str(i)+'_'+filename, y_test, delimiter=',')
                
                #savetxt('splits/test_index_'+str(i)+'.csv', test_index, delimiter=',')
                
                # create validation dataset for early stopping
                #strat = np.sign(y_train)
                #valid_split = ValidSplit(cv=0.1, stratified=strat, random_state=42)

                # train
                rs.fit(X_train, y_train)
                
                # predict
                #scaler = StandardScaler()
                #X_train = scaler.fit_transform(X_train)
                #X_test = scaler.transform(X_test)
                print(rs.best_estimator_.predict(X_train))
                best_preds_train = rs.best_estimator_.predict(X_train)
                best_preds_test = rs.best_estimator_.predict(X_test)

                #print(best_preds_test, type(best_preds_train))
                #print(best_preds_test, type(best_preds_test))
                # predict survival function
                # d = predict_survival_function(X_test, dataframe=True)

                # save predictions, get dataset name
                savetxt('predictions/train_preds_'+str(i)+'_'+filename, best_preds_train, delimiter=',')
                savetxt('predictions/test_preds_'+str(i)+'_'+filename, best_preds_test, delimiter=',')
                
                # save hyperparameter settings
                params = rs.best_estimator_.get_params
                best_params['best_params_'+dataset_name] += [rs.best_params_]
                best_model['best_model_'+dataset_name] += [params]
                #print(rs.best_params_)
                # save c-index values
                try:
                        score_train = cindex_censored(y_train, best_preds_train.reshape(-1))
                        score_test = cindex_censored(y_test, best_preds_test.reshape(-1))
                        outer_scores['cindex_train_'+dataset_name] += [score_train]
                        outer_scores['cindex_test_'+dataset_name] += [score_test]
                except:
                        outer_scores['cindex_train_'+dataset_name] += [np.nan]
                        outer_scores['cindex_test_'+dataset_name] += [np.nan]
        df_best_params = pd.DataFrame(best_params)
        df_best_model = pd.DataFrame(best_model)
        df_outer_scores = pd.DataFrame(outer_scores)
        df_metrics = pd.concat([df_best_params,df_best_model,df_outer_scores], axis=1)
        df_metrics.to_csv('metrics/metric_summary_'+str(i)+'_'+filename, index=False)
        return best_model, best_params, outer_scores, best_preds_train, best_preds_test, X_train, X_test, y_train, y_test
                

In [8]:
data = load_support(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
X  = data.data
X.dtypes
print(data.target.shape)
# d = data.target.shape[0]
# #y = np.hstack((data.target.reshape(d,1),data.target.reshape(d,1)))
# #y.shape
# type(data.target)
pd.concat([data.target, data.target],axis=1)

(8873,)


,target,target
0,3.0,3.0
1,3.0,3.0
2,3.0,3.0
3,3.0,3.0
4,3.0,3.0
...,...,...
8868,-2029.0,-2029.0
8869,-2029.0,-2029.0
8870,-2029.0,-2029.0
8871,-2029.0,-2029.0


In [63]:
data_set_fns = [load_metabric] #, load_flchain, load_rgbsg, load_support, load_tcga]
for dataset in data_set_fns:
    # get name of current dataset
    data = dataset(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
    X  = data.data #.astype(np.float32)
    y = data.target
    #y = pd.concat([data.target, data.target],axis=1)
    #y.columns = ['target_1','target_2']
    print('Dataset:',data.filename)
    print(X.dtypes)
    print(y.dtypes)
    
    net = NeuralNet(
    SurvivalModel, 
    module__n_layers = 1,
    module__in_features = X.shape[1],
    #module__num_nodes = 32,
    #module__dropout = 0.1, # these could also be removed
    module__out_features = y.shape,
    # for split sizes when result size = 1
    iterator_train__drop_last=True,
    #iterator_valid__drop_last=True,
    criterion=AFTLoss,
    optimizer=torch.optim.AdamW,
    optimizer__weight_decay = 0.4,
    batch_size=32, # separate train and valid->iterator_train__batch_size=128 and iterator_valid__batch_size=128 ?
    #callbacks=[EarlyStopping(patience=10)],
    #TODO: enable stratification, verify
    #train_split=ValidSplit(0.1), 
    #lr=0.001,
    #max_epochs=1, #0,#100
    #train_split=None,
    verbose=1
    )
    best_model,params, outer_scores, best_preds_train, best_preds_test, X_train, X_test, y_train, y_test = train_eval(X, y, net, n_iter, data.filename)
    
#train eval function here


Dataset: METABRIC_adapted.csv
MKI67                float32
EGFR                 float32
PGR                  float32
ERBB2                float32
hormone_treatment    float32
radiotherapy         float32
chemotherapy         float32
ER_positive          float32
age                  float32
dtype: object
float32
<generator object _BaseKFold.split at 0x7feec235d230>
(1522, 9) <class 'pandas.core.frame.DataFrame'>
(1522,) <class 'pandas.core.series.Series'>
(381, 9) (381, 9)
(381,) (381,)
<generator object _BaseKFold.split at 0x7fee70b45150>


ValueError: Invalid parameter 'lr' for estimator Pipeline(steps=[('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('customstandardscaler',
                                                  CustomStandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f81f887e590>),
                                                 ('customstandardscaler2',
                                                  CustomStandardScaler2(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f81f887ed40>)])),
                ('estimator',
                 <class 'skorch.net.NeuralNet'>[uninitialized](
  module=<class '__main__.SurvivalModel'>,
  module__in_features=9,
  module__n_layers=1,
  module__out_features=(1903,),
))]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [11]:
from skorch import NeuralNetRegressor

In [14]:
data_set_fns = [load_metabric] #, load_flchain, load_rgbsg, load_support, load_tcga]
for dataset in data_set_fns:
    # get name of current dataset
    data = dataset(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
    X  = data.data #.astype(np.float32)
    y = data.target
    y = pd.concat([data.target, data.target],axis=1)
    y.columns = ['target_1','target_2']
    print('Dataset:',data.filename)
    print(X.dtypes)
    print(y.dtypes)
    print('y shape', y.shape)
    
    net = NeuralNet(
    SurvivalModel, 
    module__n_layers = 1,
    module__in_features = X.shape[1],
    #module__num_nodes = 32,
    #module__dropout = 0.1, # these could also be removed
    module__out_features = y.shape[1],
    # for split sizes when result size = 1
    iterator_train__drop_last=True,
    #iterator_valid__drop_last=True,
    criterion=EHLoss,
    optimizer=torch.optim.AdamW,
    optimizer__weight_decay = 0.4,
    batch_size=32, # separate train and valid->iterator_train__batch_size=128 and iterator_valid__batch_size=128 ?
    #callbacks=[EarlyStopping(patience=10)],
    #TODO: enable stratification, verify
    #train_split=ValidSplit(0.1), 
    #lr=0.001,
    #max_epochs=1, #0,#100
    #train_split=None,
    verbose=1
    )

    pipe = Pipeline(
    [
    ('estimator', net)]
    )
    # pipe = Pipeline([
    # ('scale', StandardScaler()),
    # ('estimator', net),
    # ])
    rs = RandomizedSearchCV(pipe, param_grid_breslow, n_jobs=-1, scoring =  'neg_mean_squared_error', n_iter=n_iter, refit=True)
    

    # train
    rs.fit(X, y)
    
    # predict
    #scaler = StandardScaler()
    #X_train = scaler.fit_transform(X_train)
    #X_test = scaler.transform(X_test)
    print(rs.best_estimator_.predict(X_train))
    best_preds_train = rs.best_estimator_.predict(X_train)
    best_preds_test = rs.best_estimator_.predict(X_test)


Dataset: METABRIC_adapted.csv
MKI67                float32
EGFR                 float32
PGR                  float32
ERBB2                float32
hormone_treatment    float32
radiotherapy         float32
chemotherapy         float32
ER_positive          float32
age                  float32
dtype: object
target_1    float32
target_2    float32
dtype: object
y shape (1903, 2)


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/skorch/net.py", line 1230, in fit
    self.partial_fit(X, y, **fit_params)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/skorch/net.py", line 1189, in partial_fit
    self.fit_loop(X, 

In [ ]:
import sklearn
sklearn.metrics.get_scorer_names() 

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [26]:
# Define Scorer
def custom_scoring_function(y_true, y_pred):

        #y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, np.ndarray):
            y_pred = torch.from_numpy(y_pred)
        if isinstance(y_true, np.ndarray):
            y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, pd.Series):
            y_pred = torch.tensor(y_pred.values)
        if isinstance(y_true, pd.Series):
            y_true = torch.tensor(y_true.values)
        score = eh_likelihood_torch_2(y_true, y_pred)
        print('score',score)
        return score.numpy()

scoring_function = make_scorer(custom_scoring_function, greater_is_better=False)

In [61]:
import numpy as np
from sklearn.datasets import make_regression
import torch
from torch import nn
import torch.nn.functional as F


# This is a toy dataset for binary classification, 1000 data points with 20 features each
X, y = make_regression(1000, 20, n_informative=10, random_state=0, n_targets=2)
X_old, y_old = X.astype(np.float32), y.astype(np.float32)

data = load_metabric(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
X  = data.data #.iloc[:1000]#.astype(np.float32)
y = data.target
y = pd.concat([data.target, data.target],axis=1) #.iloc[:1000]

    
class SurvivalModel(nn.Module):
    def __init__(self, n_layers, in_features, num_nodes, dropout, out_features):
        super(SurvivalModel, self).__init__()
        self.n_layers = n_layers
        self.in_features = in_features
        self.num_nodes = num_nodes
        self.dropout = dropout
        self.out_features = out_features
        model = []
        # first layer
        model.append(torch.nn.Linear(in_features, num_nodes))
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Dropout(dropout))
        model.append(torch.nn.BatchNorm1d(num_nodes))

        for i in range(n_layers-1):
            model.append(torch.nn.Linear(num_nodes, num_nodes))
            #init.kaiming_normal_(model[-1].weight, nonlinearity='relu')
            model.append(torch.nn.ReLU())
            model.append(torch.nn.Dropout(dropout))
            model.append(torch.nn.BatchNorm1d(num_nodes))

        # output layer
        model.append(torch.nn.Linear(num_nodes, out_features))
    
        self.layers = nn.Sequential(*model)

        # for layer in self.layers:
        #     if isinstance(layer, nn.Linear):
        #         #nn.init.uniform_(layer.weight, a=-0.5, b=0.5)
        #         nn.init.kaiming_normal_(layer.weight)


    def forward(self, x):
        res = self.layers(x)
        print('res',res)
        return res
    


from skorch import NeuralNetRegressor

net = NeuralNetRegressor(
    SurvivalModel,
    max_epochs=20,
    module__n_layers = 1,
    module__in_features = X.shape[1],
    module__num_nodes = 32,
    module__dropout = 0.1, # these could also be removed
    module__out_features = y.shape[1],
    criterion=AFTLoss,
    # for split sizes when result size = 1
    iterator_train__drop_last=True,
    lr=0.001,
#     device='cuda',  # uncomment this to train with CUDA
)
param_grid_breslow = {'lr':[0.01]}
rs = RandomizedSearchCV(net, param_grid_breslow, n_jobs=-1, scoring = 'neg_mean_absolute_error', n_iter=n_iter, refit=True)
    
#scoring_function
    # train
rs.fit(X.values, y.values)

# Training the network
#net.fit(X, y)






res tensor([[ 1.3605e-01, -4.0536e-02],
        [-2.6749e-01, -4.3811e-01],
        [ 1.6736e-01, -1.9756e-01],
        [ 2.5982e-01,  1.9835e-01],
        [ 2.3864e-01, -3.8675e-01],
        [ 8.8213e-02,  1.0269e-01],
        [-1.2194e-01,  2.0251e-01],
        [ 9.0117e-02,  4.6018e-02],
        [-2.5874e-01, -7.0655e-02],
        [ 1.7664e-01, -2.2835e-02],
        [-5.2692e-03, -6.0187e-02],
        [ 4.5565e-02, -7.2283e-02],
        [ 1.3050e-01, -2.2808e-01],
        [ 1.4961e-01,  3.1284e-02],
        [ 2.0955e-01, -2.4668e-01],
        [ 3.5213e-01,  1.6539e-01],
        [ 1.3855e-01, -2.5153e-02],
        [-2.1017e-02,  6.5905e-02],
        [ 7.2905e-01,  6.8269e-02],
        [ 1.3487e-01, -1.9563e-01],
        [ 1.7411e-01,  1.3675e-02],
        [ 2.0898e-01,  7.6645e-02],
        [ 1.6142e-01, -9.6822e-03],
        [ 2.5252e-02, -7.1118e-02],
        [ 7.2353e-01, -4.9765e-01],
        [ 1.7573e-01, -4.6562e-04],
        [-1.4910e-01,  8.4651e-01],
        [ 1.6982e-01,  2

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/skorch/regressor.py", line 91, in fit
    return super(NeuralNetRegressor, self).fit(X, y, **fit_params)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/skorch/net.py", line 1230, in fit
    self.partial_fit(X, y, **fit_params)
  File "/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/skorch/net.py", line 1189, in partial_fit
    self.fit_loo

ictor type <class 'torch.Tensor'>
tensor(9.2232) tensor(8.5503, grad_fn=<DivBackward0>) res tensor(-1.5981, grad_fn=<DivBackward0>) tensor(9.4372) tensor(-0.2968, grad_fn=<DivBackward0>)
tensor(9.1091, grad_fn=<DivBackward0>) loss tensor(-1.3090, grad_fn=<DivBackward0>) tensor(0.6284, grad_fn=<NegBackward0>)
tensor(-0.3301, grad_fn=<DivBackward0>)
loss tensor(0.6507, grad_fn=<NegBackward0>)
tensor([[ 1.1177e-01, -1.0263e-01],
        [ 6.6378e-01,  1.2264e-01],
        [ 2.0045e-01,  4.4903e-01],
        [ 9.9885e-01,  4.2749e-01],
        [-2.6593e-01, -3.9031e-01],
        [-5.8369e-02,  2.5767e-01],
        [-1.9582e+00,  4.9075e-01],
        [ 9.1617e-01,  1.1869e-01],
        [ 5.7928e-01, -2.6876e-01],
        [-2.8740e-01,  1.8818e-02],
        [ 7.6196e-02, -1.2846e-01],
        [ 7.3036e-01,  1.1969e-01],
        [-4.8280e-01, -2.5260e-01],
        [ 2.2954e-02, -2.1771e-01],
        [-5.5487e-01, -2.4247e-01],
        [ 5.6256e-01,  7.9699e-02],
        [-5.2144e-01, -1.7109e

5.8065e-01,  1.4453e-01],
        [-1.1345e+00,  6.0600e-02],
        [ 1.0419e+00, -4.9611e-02],
        [-1.3106e-01,  2.1022e-01],
        [-3.8234e-01,  2.3816e-01],
        [-2.1864e+00,  5.8654e-02],
        [ 2.5890e-01,  2.3628e-01],
        [-1.7154e+00, -2.0312e-01],
        [-2.7808e+00,  1.0340e-01],
        [ 1.0903e-01,  3.1776e-01],
        [-9.6534e-01, -2.6735e-01],
        [-8.6385e-01, -6.7720e-02],
        [ 6.7001e-01, -9.1338e-02],
        [ 2.4555e-01, -4.9717e-02],
        [-1.2007e+00,  7.3372e-02],
        [-1.6577e-01,  2.6373e-01],
        [ 3.2777e-01,  2.5255e-01],
        [ 1.9776e-01, -2.0660e-01],
        [-5.5177e-01,  4.5996e-02],
        [-1.1964e+00, -3.5423e-01],
        [-4.2305e-01, -1.4762e-01],
        [ 3.9070e-02,  3.9144e-01],
        [ 3.4601e-01,  2.4604e-01],
        [-9.2957e-02,  1.9043e-01],
        [-1.5754e+00, -1.0251e-01],
        [ 1.1326e+00, -1.6381e-01],
        [-9.6148e-01, -9.7884e-05],
        [-1.2273e+00, -1.2824e-01],
  

In [42]:
y_old.shape

(1000, 2)

In [43]:
X_old.shape

(1000, 20)

In [47]:
X.shape

(1000, 9)

In [48]:
y.shape

(1000, 2)